# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Libraries
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [2]:
# your code here
admissions_predict = pd.read_csv(r"C:\Users\mari_\IronHack\W4\lab-advanced-pandas\your-code\data\Admission_Predict.csv")

Remove trailing spaces at the end of the column names if there are any.


In [3]:
# your code here

admissions_predict.columns= ['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit']
admissions_predict.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Let's evaluate the dataset by looking at the `head` function.

In [4]:
# your code here
admissions_predict.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [5]:
# your code here
#This shows that there are no null or na values on the dataframe

admissions_predict.isnull().sum()
admissions_predict.isna().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [6]:
# your code here

admissions_predict.set_index("Serial No.", drop = False, inplace = True)

#admissions_predict


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [7]:
a = sum(admissions_predict[['GRE Score', 'CGPA']].duplicated())


In [8]:
admissions_predict.shape

(385, 9)

In [9]:
sum(admissions_predict["GRE Score"].duplicated())

336

In [10]:
# your code here
#This shows that there are no values that have the same GRE Score and CGPA on the same row.
print("The value of diferent values for GRE Score and CGPA is: ",sum(admissions_predict[["GRE Score", "CGPA"]].duplicated()))
print("The value of diferent values for GRE Score is: ",len(admissions_predict["GRE Score"].duplicated()))

The value of diferent values for GRE Score and CGPA is:  0
The value of diferent values for GRE Score is:  385


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [11]:
admissions_predict.set_index(["GRE Score", "CGPA"], drop = True, inplace = True)

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [12]:
# your code here
admissions_predict.reset_index(inplace = True)


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [13]:
# your code here
filter = (admissions_predict["CGPA"] > 9) & (admissions_predict["Research"] == 1)
print( "The number of rows is:", filter.sum())

The number of rows is: 101


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [14]:
# your code here
filter1 = (admissions_predict["CGPA"] > 9) & (admissions_predict["SOP"] < 3.5) #& (admissions_predict["Chance of Admit"].mean())

new = admissions_predict.assign( A = filter1)

sum_nal = new.groupby("A", as_index=True).agg({"Chance of Admit": "mean"})
sum_nal

#The mean chance of admit for these applicants is 0.8. 

,Chance of Admit
A,
False,0.722895
True,0.802000


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [15]:
def standardize(col):
        return pd.Series(col-np.mean(col)) / np.std(col)
        
       




Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [16]:
# your code here


admissions_predict["CGPA_std"] = standardize(admissions_predict["CGPA"])
admissions_predict["GRE_std"] = standardize(admissions_predict["GRE Score"])
admissions_predict["LOR_std"] = standardize(admissions_predict["LOR"])
admissions_predict

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338
5,321,8.20,6,109,3,3.0,4.0,1,0.75,-0.660055,0.369672,0.635019
6,308,7.90,7,101,2,3.0,4.0,0,0.68,-1.158723,-0.756445,0.635019
7,302,8.00,8,102,1,2.0,1.5,0,0.50,-0.992501,-1.276191,-2.155874
8,323,8.60,9,108,3,3.5,3.0,0,0.45,0.004836,0.542921,-0.481338
9,327,9.00,10,111,4,4.0,4.5,1,0.84,0.669726,0.889419,1.193197


We will generate the decision choice at random using the code below. Please run the cell.

In [17]:
# Libraries
from random import choices

In [18]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions_predict.shape[0])
#decision_choice

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [19]:
# your code here
admissions_predict["deciding_column"] = admissions_predict.lookup(admissions_predict.index, decision_choice)


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [20]:
# your code here
admissions_predict["decision"] = [1 if x > 0.8 else 0 for x in admissions_predict["deciding_column"]]

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [21]:
# your code here
total_applicants = admissions_predict["decision"].sum()
total_applicants

81

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [22]:
# your code here

admissions_predict = admissions_predict.rename(columns = lambda x: x.strip().strip().replace(' ', '_').lower())

In [23]:
admissions_predict

,gre_score,cgpa,serial_no.,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.750174,1
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.992501,0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433,0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,-0.481338,0
5,321,8.20,6,109,3,3.0,4.0,1,0.75,-0.660055,0.369672,0.635019,0.635019,0
6,308,7.90,7,101,2,3.0,4.0,0,0.68,-1.158723,-0.756445,0.635019,-0.756445,0
7,302,8.00,8,102,1,2.0,1.5,0,0.50,-0.992501,-1.276191,-2.155874,-2.155874,0
8,323,8.60,9,108,3,3.5,3.0,0,0.45,0.004836,0.542921,-0.481338,0.542921,0
9,327,9.00,10,111,4,4.0,4.5,1,0.84,0.669726,0.889419,1.193197,0.889419,1


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [34]:
# your code here
adjusted_gre = pd.cut(np.where(admissions_predict["university_rating"]>=4, admissions_predict["gre_score"]+10, admissions_predict["gre_score"]),bins=4)
adjusted_gre

[(335.0, 350.0], (305.0, 320.0], (320.0, 335.0], (305.0, 320.0], (335.0, 350.0], ..., (320.0, 335.0], (320.0, 335.0], (335.0, 350.0], (305.0, 320.0], (335.0, 350.0]]
Length: 385
Categories (4, interval[float64]): [(289.94, 305.0] < (305.0, 320.0] < (320.0, 335.0] < (335.0, 350.0]]